In [ ]:
import re
from datetime import datetime, timedelta
import tkinter as tk
from tkinter import ttk, messagebox, simpledialog
import csv
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt

class ExpenseTrackerApp(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Expense Tracker")
        self.geometry("1300x600")
        screen_width = self.winfo_screenwidth()
        screen_height = self.winfo_screenheight()
        self.geometry(f"{screen_width}x{screen_height}")
        self.expenses = []
        self.categories = [
            "Food",
            "Transportation",
            "Utilities",
            "Entertainment",
            "Other",
        ]
        self.category_var = tk.StringVar(self)
        self.category_var.set(self.categories[0])
        self.currencies = ["USD", "EUR", "GBP", "JPY", "INR"]
        self.create_widgets()

    def create_widgets(self):
        self.label = tk.Label(
            self, text="Expense Tracker", font=("Helvetica", 20, "bold")
        )
        self.label.pack(pady=10)
        self.frame_input = tk.Frame(self)
        self.frame_input.pack(pady=10)
        self.expense_label = tk.Label(
            self.frame_input, text="Expense Amount:", font=("Helvetica", 12)
        )
        self.expense_label.grid(row=0, column=0, padx=5)
        self.expense_entry = tk.Entry(
            self.frame_input, font=("Helvetica", 12), width=15
        )
        self.expense_entry.grid(row=0, column=1, padx=5)
        self.item_label = tk.Label(
            self.frame_input, text="Item Description:", font=("Helvetica", 12)
        )
        self.item_label.grid(row=0, column=2, padx=5)
        self.item_entry = tk.Entry(self.frame_input, font=("Helvetica", 12), width=20)
        self.item_entry.grid(row=0, column=3, padx=5)
        self.category_label = tk.Label(
            self.frame_input, text="Category:", font=("Helvetica", 12)
        )
        self.category_label.grid(row=0, column=4, padx=5)
        self.category_dropdown = ttk.Combobox(
            self.frame_input,
            textvariable=self.category_var,
            values=self.categories,
            font=("Helvetica", 12),
            width=15,
        )
        self.category_dropdown.grid(row=0, column=5, padx=5)


        self.date_label = tk.Label(
            self.frame_input, text="Date (YYYY-MM-DD):", font=("Helvetica", 12)
        )
        self.date_label.grid(row=0, column=6, padx=5)


        self.date_entry = tk.Entry(self.frame_input, font=("Helvetica", 12), width=15)
        self.date_entry.grid(row=0, column=7, padx=5)

        self.add_button = tk.Button(self, text="Add Expense", command=self.add_expense)
        self.add_button.pack(pady=5)
        self.frame_list = tk.Frame(self)
        self.frame_list.pack(pady=10)
        self.scrollbar = tk.Scrollbar(self.frame_list)
        self.scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
        self.expense_listbox = tk.Listbox(
            self.frame_list,
            font=("Helvetica", 12),
            width=70,
            yscrollcommand=self.scrollbar.set,
        )
        self.expense_listbox.pack(pady=5)
        self.scrollbar.config(command=self.expense_listbox.yview)
        self.edit_button = tk.Button(
            self, text="Edit Expense", command=self.edit_expense
        )
        self.edit_button.pack(pady=5)
        self.delete_button = tk.Button(
            self, text="Delete Expense", command=self.delete_expense
        )
        self.delete_button.pack(pady=5)
        self.save_button = tk.Button(
            self, text="Save Expenses", command=self.save_expenses
        )
        self.save_button.pack(pady=5)
        self.total_label = tk.Label(
            self, text="Total Expenses:", font=("Helvetica", 12)
        )
        self.total_label.pack(pady=5)
        self.show_chart_button = tk.Button(
            self, text="Show Expenses Chart", command=self.show_expenses_chart
        )
        self.show_chart_button.pack(pady=5)

        #date
        self.get_expenses_button = tk.Button(
        self, text="Get Expenses by Date", command=self.get_expenses_by_date)
        self.get_expenses_button.pack(pady=5)

        #week
        self.get_expenses_week_button = tk.Button(
        self, text="Get Expenses by Week", command=self.get_expenses_by_week)
        self.get_expenses_week_button.pack(pady=5)

        self.update_total_label()


    
    
    def add_expense(self):
        expense = self.expense_entry.get()
        item = self.item_entry.get()
        category = self.category_var.get()
        date = self.date_entry.get()
        if expense and date:
            if not expense.replace(".", "", 1).isdigit():
                messagebox.showwarning("Warning", "Expense amount must be a number.")
                return
            if not self.validate_date(date):
                messagebox.showwarning("Warning", "Invalid date format. Please use YYYY-MM-DD.")
                return
            self.expenses.append((expense, item, category, date))
            self.expense_listbox.insert(
                tk.END, f"{expense} - {item} - {category} ({date})"
            )
            self.expense_entry.delete(0, tk.END)
            self.item_entry.delete(0, tk.END)
            self.date_entry.delete(0, tk.END)
        else:
            messagebox.showwarning("Warning", "Expense and Date cannot be empty.")
        self.update_total_label()

    #week code
    def get_expenses_by_week(self):
    # Prompt the user to input start date and end date for the week
        start_date = simpledialog.askstring("Enter Start Date", "Enter the start date of the week (YYYY-MM-DD):")
        end_date = simpledialog.askstring("Enter End Date", "Enter the end date of the week (YYYY-MM-DD):")
        if start_date and end_date:
            try:
                # Convert input strings to datetime objects
                start_date = datetime.strptime(start_date, "%Y-%m-%d")
                end_date = datetime.strptime(end_date, "%Y-%m-%d")
                # Calculate the end of the week (assuming Sunday is the end of the week)
                end_of_week = start_date + timedelta(days=(6 - start_date.weekday()))
                if end_of_week > end_date:
                    end_of_week = end_date
                matched_expenses = []
                total_expenses = 0
                # Filter expenses for the specified week and calculate total expenses
                for expense in self.expenses:
                    expense_date = datetime.strptime(expense[3], "%Y-%m-%d")
                    if start_date <= expense_date <= end_of_week:
                        matched_expenses.append(expense)
                        total_expenses += float(expense[0])
                # Display the total expenses for the specified week
                if matched_expenses:
                    messagebox.showinfo("Expenses for Week", f"Total Expenses for {start_date.strftime('%Y-%m-%d')} to {end_of_week.strftime('%Y-%m-%d')}: RS {total_expenses:.2f}\n\n" + "\n".join([f"{exp[0]} - {exp[1]} - {exp[2]}" for exp in matched_expenses]))
                else:
                    messagebox.showinfo("No Expenses", "No expenses recorded for the specified week.")
            except ValueError:
                messagebox.showwarning("Invalid Dates", "Invalid date format or range. Please use YYYY-MM-DD format and ensure start date is before end date.")


    def validate_date(self, date_text):
        pattern = r'^\d{4}-\d{2}-\d{2}$'
        if not re.match(pattern, date_text):
            return False
        try:
            year, month, day = map(int, date_text.split('-'))
            return 1 <= month <= 12 and 1 <= day <= 31
        except ValueError:
            return False


    def edit_expense(self):
        selected_index = self.expense_listbox.curselection()
        if selected_index:
            selected_index = selected_index[0]
            selected_expense = self.expenses[selected_index]
            new_expense = simpledialog.askstring(
                "Edit Expense", "Enter new expense:", initialvalue=selected_expense[0]
            )
            if new_expense:
                self.expenses[selected_index] = (
                    new_expense,
                    selected_expense[1],
                    selected_expense[2],
                    selected_expense[3],
                )
                self.refresh_list()
                self.update_total_label()

    def get_expenses_by_date(self):
    # Prompt the user to input a date
        input_date = simpledialog.askstring("Enter Date", "Enter the date (YYYY-MM-DD):")
        if input_date:
            matched_expenses = []
            total_expenses = 0
        # Filter expenses for the specified date and calculate total expenses
            for expense in self.expenses:
                if expense[3] == input_date:
                    matched_expenses.append(expense)
                    total_expenses += float(expense[0])
        # Display the total expenses for the specified date
            if matched_expenses:
                messagebox.showinfo("Expenses on " + input_date, f"Total Expenses: RS {total_expenses:.2f}\n\n" + "\n".join([f"{exp[0]} - {exp[1]} - {exp[2]}" for exp in matched_expenses]))
            else:
                messagebox.showinfo("No Expenses", "No expenses recorded for the specified date.")


    def delete_expense(self):
        selected_index = self.expense_listbox.curselection()
        if selected_index:
            selected_index = selected_index[0]
            del self.expenses[selected_index]
            self.expense_listbox.delete(selected_index)
            self.update_total_label()

    def refresh_list(self):
        self.expense_listbox.delete(0, tk.END)
        for expense, item, category, date in self.expenses:
            self.expense_listbox.insert(
                tk.END, f"{expense} - {item} - {category} ({date})")

    def update_total_label(self):
        total_expenses = sum(float(expense[0]) for expense in self.expenses)
        self.total_label.config(text=f"Total Expenses: RS {total_expenses:.2f}")
        
    def save_expenses(self):
        with open("expenses.csv", "w", newline="") as csvfile:
            writer = csv.writer(csvfile)
            column_headers = ["Expense Amount", "Item Description", "Category", "Date"]
            writer.writerow(column_headers)
            for expense in self.expenses:
                writer.writerow(expense)

    def show_expenses_chart(self):
        category_totals = {}
        for expense, _, category, _ in self.expenses:
            try:
                amount = float(expense)
            except ValueError:
                continue
            category_totals[category] = category_totals.get(category, 0) + amount
        categories = list(category_totals.keys())
        expenses = list(category_totals.values())
        plt.figure(figsize=(8, 6))
        plt.pie(
            expenses, labels=categories, autopct="%1.1f%%", startangle=140, shadow=True
        )
        plt.axis("equal")
        plt.title(f"Expense Categories Distribution (RS)")
        plt.show()

if __name__ == "__main__":
    app = ExpenseTrackerApp()
    app.mainloop()
